Basic LSTM

In [1]:
import numpy as np
import pandas as pd
import string
import tensorflow as tf
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence, text
import json
import sklearn
from sklearn import preprocessing as skpp

Using TensorFlow backend.


In [2]:
data = pd.read_csv('./dataset/cleaned_lyrics.csv')

In [ ]:
print(data)

# cols: index, song, year, artist, genre, lyrics
# N = 227449 songs

In [3]:
numpy_data = data['lyrics'].values
max_words = 5000

# create a new Tokenizer
tokenizer = text.Tokenizer(num_words=max_words, oov_token='<UNK>')
# feed our tweets to the Tokenizer
tokenizer.fit_on_texts(numpy_data)

# Tokenizers come with a convenient list of words and IDs
dictionary = tokenizer.word_index

with open('dictionary.json', 'w') as dictionary_file:
    json.dump(dictionary, dictionary_file)

In [ ]:
indexed_data = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for txt in numpy_data:
    wordIndices = [dictionary[word] for word in text.text_to_word_sequence(txt)]
    indexed_data.append(wordIndices)

# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
indexed_data = np.asarray(indexed_data)

In [5]:
tokenizer.word_index = {e:i for e,i in tokenizer.word_index.items() if i <= max_words} # <= because tokenizer is 1 indexed
tokenizer.word_index[tokenizer.oov_token] = max_words + 1
indexed_data = tokenizer.texts_to_sequences(numpy_data)
indexed_data = np.array(indexed_data)

print(indexed_data[0])

[39, 61, 74, 3, 517, 3, 27, 15, 81, 534, 80, 4, 1, 1279, 105, 928, 139, 184, 24, 7, 247, 2, 25, 4, 110, 13, 2, 49, 2382, 18, 6, 967, 2581, 3, 27, 257, 87, 967, 159, 3, 3, 78, 7, 30, 14, 61, 218, 40, 210, 3, 22, 86, 4, 173, 146, 335, 100, 51, 1, 1454, 18, 205, 56, 3, 4, 195, 7, 74, 3, 78, 2, 3500, 247, 1016, 99, 6, 231, 910, 111, 125, 3, 38, 1, 1007, 4, 8, 90, 24, 3, 85, 81, 86, 10, 203, 790, 3, 322, 32, 8, 316, 5, 195, 7, 1425, 3, 285, 27, 49, 187, 28, 86, 18, 7, 4, 355, 30, 102, 231, 30, 102, 765, 30, 102, 428, 10, 127, 1083, 30, 102, 163, 30, 102, 1192, 73, 278, 25, 29, 100, 73, 37, 65, 10, 32, 73, 38, 6, 231, 2758, 488, 6, 5001, 2758, 2, 26, 118, 231, 2758, 30, 102, 163, 73, 224, 25, 29, 100, 73, 37, 65, 10, 32, 3280, 15, 3555, 80, 35, 2, 22, 931, 3, 37, 170, 23, 7, 97, 3, 89, 57, 1, 746, 105, 173, 10, 5001, 2, 173, 10, 5001, 3, 1517, 33, 3, 137, 14, 34, 15, 907, 23, 459, 2, 27, 15, 896, 3, 23, 121, 1364, 151, 556, 121, 3276, 383, 6, 1026, 30, 8, 373, 97, 269, 8, 119, 209, 3, 6, 500

In [6]:
print(indexed_data[133643])
print(len(indexed_data[133643]))
print(data['lyrics'][133643])
print(len(data['lyrics'][133643].split()))
print(dictionary['diplo'])

[528, 201, 82, 1383, 528, 201, 82, 1383, 528, 201, 82, 1383, 528, 201, 82, 1383, 528, 201, 82, 1383, 218, 44, 1226, 4, 1, 370, 200, 250, 32, 320, 9, 1, 208, 259, 32, 5, 741, 10, 471, 10, 954, 1, 712, 5, 113, 145, 3, 37, 36, 4, 153, 201, 141, 5, 33, 1, 445, 928, 19, 2262, 141, 138, 385, 4, 100, 6, 2626, 9, 29, 403, 309, 20, 154, 1, 248, 5001, 197, 141, 20, 38, 201, 664, 36, 20, 162, 38, 121, 16, 9, 6, 4325, 5001, 20, 38, 201, 664, 36, 20, 162, 38, 121, 16, 9, 6, 4325, 5001, 20, 38, 201, 3683, 20, 38, 201, 3683, 20, 38, 201, 3683, 20, 38, 201, 3683, 1662, 46, 2, 60, 9, 5, 2, 3002, 14, 1, 244, 5, 20, 1848, 1, 2568, 21, 2, 271, 3, 27, 20, 38, 95, 183, 14, 100, 3, 152, 27, 13, 33, 20, 40, 85, 152, 1120, 42, 3, 152, 926, 5, 437, 201, 32, 104, 10, 4, 201, 67, 121, 32, 67, 7, 36, 121, 1873, 185, 2013, 5, 67, 10, 985, 40, 12, 1085, 14, 320, 40, 334, 35, 1662, 2, 36, 34, 2, 36, 954, 5001, 68, 1, 200, 101, 926, 218, 44, 100, 3, 27, 20, 5001, 314, 5001, 314, 40, 320, 40, 40, 35, 2, 101, 48, 16, 6,

In [7]:
#max lyric length is 6208 at song #9467
#top 10 lyric lengths: [5131 4287 6208 3278 3167 3155 3153 2997 2750 2660]
#for top 1000 lengthiest songs, even first 1000 words seems sufficient
#for top 100 lengthiest songs, first 1500 words seems sufficient
#np.max(np.vectorize(len)(indexed_data))
#temp = np.partition(-np.vectorize(len)(indexed_data), 100)
#result_args = temp[:100]

label_encoder = skpp.LabelEncoder()
indexed_labels = np.array(label_encoder.fit_transform(data['genre'].values))
#label_encoder.inverse_transform(np.array([10, 8])) #to get original genre text back

In [14]:
num_test = 30000

#shuffle data before splitting off test set
random_indexes = np.random.permutation(len(indexed_labels))
indexed_data = indexed_data[random_indexes]
indexed_labels = indexed_labels[random_indexes]

X_train = indexed_data[:-num_test]
y_train = indexed_labels[:-num_test]
X_test  = indexed_data[-num_test:]
y_test  = indexed_labels[-num_test:]

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [21]:
num_words = max_words + 2
# truncate and pad input sequences
max_review_length = 1000

X_train_padded = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test_padded = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:
#vocab size = 336097

In [22]:
# create the model
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(num_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(11, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train_padded, y_train, nb_epoch=1, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test_padded, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1000, 32)          160064    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 11)                1111      
Total params: 214,375
Trainable params: 214,375
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
  4288/197449 [..............................] - ETA: 1:22:08 - loss: 1.9135 - acc: 0.4513

/anaconda/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


KeyboardInterrupt: 